In [ ]:
from numpy import linspace, zeros
from matplotlib.pyplot import subplots
from weichsel_schwarz import Network

In [ ]:
capping_rate_const_row = linspace(0.01, 1.0, 20)
num_replicates = 10
order_parameter_mat = zeros((capping_rate_const_row.size, num_replicates))
rel_growth_rate_mat = zeros((capping_rate_const_row.size, num_replicates))
nucleation_rate_mat = zeros((capping_rate_const_row.size, num_replicates))
density_active_mat = zeros((capping_rate_const_row.size, num_replicates))

In [ ]:
from joblib import Parallel, delayed
def compute_network_properties(x):
    network = Network(capping_rate_const=x)
    network.simulate()
    network.compute_order_parameter()
    rel_growth_rate = network.leading_edge_position / network.current_time / network.elongation_rate_const
    nucleation_rate = (network.barbed_position_mat.shape[0] - 150) / network.current_time
    density_active = network.active_filament_index.size
    return network.order_parameter, rel_growth_rate, nucleation_rate, density_active

In [ ]:
for i in range(num_replicates):
    i_output = Parallel(n_jobs=-1)(delayed(compute_network_properties)(j) for j in capping_rate_const_row)
    for k in range(len(i_output)):
        order_parameter_mat[k, i] = i_output[k][0]
        rel_growth_rate_mat[k, i] = i_output[k][1]
        nucleation_rate_mat[k, i] = i_output[k][2]
        density_active_mat[k, i] = i_output[k][3]

In [ ]:
def format_figure(fig_hand, axes_hand):
    fig_hand.set_figheight(4)
    fig_hand.set_figwidth(4)
    axes_hand.tick_params(labelsize=12, length=6, width=1)

In [ ]:
fig1_hand, axes1_hand = subplots()
axes1_hand.errorbar(capping_rate_const_row, order_parameter_mat.mean(axis=1), order_parameter_mat.std(axis=1), linewidth=2)
axes1_hand.set_xlabel('Capping rate constant', fontsize=12)
axes1_hand.set_ylabel('Order parameter', fontsize=12)
format_figure(fig1_hand, axes1_hand)

In [ ]:
fig2_hand, axes2_hand = subplots()
axes2_hand.errorbar(capping_rate_const_row, rel_growth_rate_mat.mean(axis=1), rel_growth_rate_mat.std(axis=1), linewidth=2)
axes2_hand.set_xlabel('Capping rate constant', fontsize=12)
axes2_hand.set_ylabel('Normalized growth rate', fontsize=12)
format_figure(fig2_hand, axes2_hand)

In [ ]:
fig4_hand, axes4_hand = subplots()
axes4_hand.errorbar(capping_rate_const_row, nucleation_rate_mat.mean(axis=1), nucleation_rate_mat.std(axis=1), linewidth=2)
axes4_hand.set_ylim(0, 33)
axes4_hand.set_xlabel('Capping rate constant', fontsize=12)
axes4_hand.set_ylabel('Nucleation rate', fontsize=12)
format_figure(fig4_hand, axes4_hand)

In [ ]:
fig5_hand, axes5_hand = subplots()
axes5_hand.errorbar(capping_rate_const_row, density_active_mat.mean(axis=1), nucleation_rate_mat.std(axis=1), linewidth=2)
axes5_hand.set_ylim(0, 12)
axes5_hand.set_xlabel('Capping rate constant', fontsize=12)
axes5_hand.set_ylabel('Number of active filaments', fontsize=12)
format_figure(fig5_hand, axes5_hand)